In [ ]:
!pip install clean-text
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.0 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=b2acdeb79ca8bddf21d684a14aee6a09b93a40d74ede2e0469366a1f89ea9b99
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import nltk # Natural Language Tool Kit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix
import re # for regular expression - to clean data i.e. to remove special characters, emojis
from cleantext import clean
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

**Data Exploration and Preprocessing**

In [ ]:
#load the dataset
data=pd.read_csv("blogs.csv")

In [ ]:
data

,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism
...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc
1996,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc
1997,Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...,talk.religion.misc
1998,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc


In [ ]:
data.shape

(2000, 2)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    2000 non-null   object
 1   Labels  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [ ]:
data.dtypes

,0
Data,object
Labels,object


In [ ]:
data.describe()

,Data,Labels
count,2000,2000
unique,2000,20
top,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
freq,1,100


In [ ]:
#check the null values
data.isnull().sum()

,0
Data,0
Labels,0


In [ ]:
# EDA Rename column
data.rename(columns={'Data':'text'},inplace=True)

In [ ]:
data.head()

,text,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism


In [ ]:
#check the unique values
data['Labels'].value_counts()

,count
Labels,
alt.atheism,100
comp.graphics,100
talk.politics.misc,100
talk.politics.mideast,100
talk.politics.guns,100
soc.religion.christian,100
sci.space,100
sci.med,100
sci.electronics,100


In [ ]:
import string

def preprocessed_text(text):
    # Step 1: Convert text to lowercase
    cleaned_text = text.lower()

    # Step 2: Remove punctuation
    cleaned_text = ''.join(filter(lambda char: char not in string.punctuation, cleaned_text))

    # Step 3: Tokenize the text
    tokens = word_tokenize(cleaned_text)

    # Step 4: Remove stopwords
    stop_words = stopwords.words('english')
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Step 5: Join tokens back into a preprocessed string
    return ' '.join(filtered_tokens)



In [ ]:
# Removing Punctuation
def remove_punctuation(text):
  for punctuation in string.punctuation:
    text = text.replace(punctuation, ' ')
  return text

data["text"]=data["text"].apply(remove_punctuation)    # data after remove puncuation
data

,text,Labels
0,Path cantaloupe srv cs cmu edu magnesium club...,alt.atheism
1,Newsgroups alt atheism\nPath cantaloupe srv ...,alt.atheism
2,Path cantaloupe srv cs cmu edu das news harva...,alt.atheism
3,Path cantaloupe srv cs cmu edu magnesium club...,alt.atheism
4,Xref cantaloupe srv cs cmu edu alt atheism 53...,alt.atheism
...,...,...
1995,Xref cantaloupe srv cs cmu edu talk abortion ...,talk.religion.misc
1996,Xref cantaloupe srv cs cmu edu talk religion ...,talk.religion.misc
1997,Xref cantaloupe srv cs cmu edu talk origins 4...,talk.religion.misc
1998,Xref cantaloupe srv cs cmu edu talk religion ...,talk.religion.misc


In [ ]:
data.head()

,text,Labels
0,Path cantaloupe srv cs cmu edu magnesium club...,alt.atheism
1,Newsgroups alt atheism\nPath cantaloupe srv ...,alt.atheism
2,Path cantaloupe srv cs cmu edu das news harva...,alt.atheism
3,Path cantaloupe srv cs cmu edu magnesium club...,alt.atheism
4,Xref cantaloupe srv cs cmu edu alt atheism 53...,alt.atheism


# Tokenization

In [ ]:
# function definition
def preprocessed_text(text):
    token_list = word_tokenize(text)
    # Print the tokenized list
    print(token_list)
    print(f"Number of tokens: {len(token_list)}")

In [ ]:
# Converting to lowercase
data["text"] = data["text"].apply(str.lower)

In [ ]:
data.head()

,text,Labels
0,path cantaloupe srv cs cmu edu magnesium club...,alt.atheism
1,newsgroups alt atheism\npath cantaloupe srv ...,alt.atheism
2,path cantaloupe srv cs cmu edu das news harva...,alt.atheism
3,path cantaloupe srv cs cmu edu magnesium club...,alt.atheism
4,xref cantaloupe srv cs cmu edu alt atheism 53...,alt.atheism


In [ ]:
# Removing the stopwords
 # Using set for faster lookup
stop_words = set(nltk.corpus.stopwords.words("english"))
data["text"] = data["text"].apply(lambda x: ' '.join(filter(lambda word: word not in stop_words, x.split())))

data.head()

,text,Labels
0,path cantaloupe srv cs cmu edu magnesium club ...,alt.atheism
1,newsgroups alt atheism path cantaloupe srv cs ...,alt.atheism
2,path cantaloupe srv cs cmu edu das news harvar...,alt.atheism
3,path cantaloupe srv cs cmu edu magnesium club ...,alt.atheism
4,xref cantaloupe srv cs cmu edu alt atheism 534...,alt.atheism


**Naive Bayes Model for Text Classification**

**TF-IDF**

Term frequency works by looking at the frequency of a particular term you are concerned with relative to the document.

The highest scoring words of a document are the most relevant to that document.

In [ ]:
# Feature extraction
tf = TfidfVectorizer()
x = tf.fit_transform(data['text'])
y=data['Labels']


In [ ]:
# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Naive Bayes classifier


nb_classifier = MultinomialNB()  # Initializing the classifier
nb_classifier.fit(x_train, y_train)  # Training the classifier on the data


MultinomialNB()

In [ ]:
#size of the train data
x_train.shape,y_train.shape

((1600, 50578), (1600,))

In [ ]:
#size of the test data
x_test.shape,y_test.shape

((400, 50578), (400,))

In [ ]:
# Predictions on training data
y_prediction_train = nb_classifier.predict(x_train)
print(y_prediction_train)  # Printing the predictions


['rec.sport.baseball' 'comp.os.ms-windows.misc' 'rec.motorcycles' ...
 'rec.motorcycles' 'sci.space' 'sci.crypt']


In [ ]:
# Make predictions on testing data
y_prediction_test = nb_classifier.predict(x_test)
print(y_prediction_test)

['talk.politics.misc' 'comp.sys.ibm.pc.hardware' 'sci.med'
 'rec.sport.baseball' 'comp.sys.ibm.pc.hardware' 'sci.electronics'
 'rec.sport.baseball' 'talk.politics.mideast' 'alt.atheism' 'sci.med'
 'alt.atheism' 'sci.med' 'sci.crypt' 'rec.sport.baseball'
 'comp.sys.ibm.pc.hardware' 'comp.os.ms-windows.misc' 'rec.autos'
 'comp.graphics' 'talk.politics.guns' 'talk.politics.misc' 'misc.forsale'
 'talk.politics.misc' 'talk.religion.misc' 'rec.sport.hockey'
 'alt.atheism' 'sci.crypt' 'sci.crypt' 'rec.sport.baseball' 'rec.autos'
 'alt.atheism' 'misc.forsale' 'rec.sport.hockey' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.hockey' 'comp.sys.mac.hardware' 'sci.med'
 'rec.sport.hockey' 'rec.sport.hockey' 'comp.os.ms-windows.misc'
 'comp.sys.ibm.pc.hardware' 'talk.religion.misc' 'comp.os.ms-windows.misc'
 'comp.graphics' 'soc.religion.christian' 'rec.motorcycles' 'sci.crypt'
 'rec.motorcycles' 'alt.atheism' 'talk.politics.guns'
 'soc.religion.christian' 'sci.crypt' 'sci.space'
 'comp.os.ms-wind

#Sentiment Analysis

In [ ]:
!pip install textblob

In [ ]:
from textblob import TextBlob

# Function for sentiment analysis
def analyze_sentiment(text):
    blob = TextBlob(text)
    if blob.sentiment.polarity > 0:
        return 'Positive'
    elif blob.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

data['Sentiment_Label'] = data['text'].apply(analyze_sentiment)
data


,text,Labels,Sentiment_Label
0,path cantaloupe srv cs cmu edu magnesium club ...,alt.atheism,Positive
1,newsgroups alt atheism path cantaloupe srv cs ...,alt.atheism,Negative
2,path cantaloupe srv cs cmu edu das news harvar...,alt.atheism,Positive
3,path cantaloupe srv cs cmu edu magnesium club ...,alt.atheism,Positive
4,xref cantaloupe srv cs cmu edu alt atheism 534...,alt.atheism,Positive
...,...,...,...
1995,xref cantaloupe srv cs cmu edu talk abortion 1...,talk.religion.misc,Positive
1996,xref cantaloupe srv cs cmu edu talk religion m...,talk.religion.misc,Positive
1997,xref cantaloupe srv cs cmu edu talk origins 41...,talk.religion.misc,Positive
1998,xref cantaloupe srv cs cmu edu talk religion m...,talk.religion.misc,Positive


In [ ]:
#check the unique values of the sentiment_label column
data['Sentiment_Label'].value_counts()

,count
Sentiment_Label,
Positive,1497
Negative,500
Neutral,3


#Evaluation

In [ ]:
# Evaluate Naive Bayes classifier performance

# Compute evaluation metrics
acc = accuracy_score(y_test, y_prediction_test)
prec = precision_score(y_test, y_prediction_test, average='macro')
rec = recall_score(y_test, y_prediction_test, average='macro')
f1 = f1_score(y_test, y_prediction_test, average='macro')

# Display metrics
print(f"Accuracy: {acc*100}")
print(f"Precision: {prec*100}")
print(f"Recall: {rec*100}")
print(f"F1 Score: {f1*100}")


Accuracy: 78.75
Precision: 80.15665398581166
Recall: 79.82219039130804
F1 Score: 77.89734897625344


In [ ]:
# confusion matrix on test data
print(confusion_matrix(y_test, y_prediction_test))

[[15  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  2]
 [ 0 13  0  1  0  0  2  0  0  1  0  1  0  0  0  0  0  0  0  0]
 [ 0  0 21  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  1 21  0  0  0  0  1  0  0  1  1  0  0  0  0  0  0  0]
 [ 0  1  0  2 14  0  1  0  0  0  1  1  0  0  0  0  0  1  0  0]
 [ 0  2  6  1  0  7  1  0  2  1  1  3  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0 14  3  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 17  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  1 13  0  0  0  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 15  3  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 15  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 19  0  0  0  0  0  0  0  0]
 [ 0  1  0  2  1  0  0  1  0  0  1  0  9  1  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  1  0  0  0 15  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  1  0  0  0 19  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 23  0  0

In [ ]:
# classification report on test data
print(classification_report(y_test, y_prediction_test))

                          precision    recall  f1-score   support

             alt.atheism       0.60      0.83      0.70        18
           comp.graphics       0.72      0.72      0.72        18
 comp.os.ms-windows.misc       0.75      0.95      0.84        22
comp.sys.ibm.pc.hardware       0.75      0.84      0.79        25
   comp.sys.mac.hardware       0.88      0.67      0.76        21
          comp.windows.x       1.00      0.28      0.44        25
            misc.forsale       0.74      0.78      0.76        18
               rec.autos       0.77      0.94      0.85        18
         rec.motorcycles       0.81      0.81      0.81        16
      rec.sport.baseball       0.83      0.83      0.83        18
        rec.sport.hockey       0.65      1.00      0.79        15
               sci.crypt       0.68      1.00      0.81        19
         sci.electronics       0.75      0.56      0.64        16
                 sci.med       0.88      0.88      0.88        17
         

In [ ]:
# confusion matrix on train data
print(confusion_matrix(y_train, y_prediction_train))

[[81  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0 81  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 78  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 75  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 79  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0 74  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 82  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 82  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 84  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 81  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 85  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 81  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 83  0  1  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0  0 82  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 79  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 77  0  0

In [ ]:
# classification report on train data
print(classification_report(y_train, y_prediction_train))

                          precision    recall  f1-score   support

             alt.atheism       0.90      0.99      0.94        82
           comp.graphics       0.99      0.99      0.99        82
 comp.os.ms-windows.misc       0.99      1.00      0.99        78
comp.sys.ibm.pc.hardware       0.99      1.00      0.99        75
   comp.sys.mac.hardware       1.00      1.00      1.00        79
          comp.windows.x       1.00      0.99      0.99        75
            misc.forsale       1.00      1.00      1.00        82
               rec.autos       1.00      1.00      1.00        82
         rec.motorcycles       1.00      1.00      1.00        84
      rec.sport.baseball       1.00      0.99      0.99        82
        rec.sport.hockey       0.99      1.00      0.99        85
               sci.crypt       1.00      1.00      1.00        81
         sci.electronics       1.00      0.99      0.99        84
                 sci.med       1.00      0.99      0.99        83
         


The model was highly accurate when it predicted this class, it failed to identify a large portion of relevant instances. On the other hand, classes like rec.sport.hockey and sci.crypt achieved perfect recall (1.00), meaning all relevant instances were correctly identified, but there were some false positives, resulting in lower precision.

The overall accuracy of the model on the test set was around  0.78, signifying that 78% of the instances were correctly classified.

the Naive Bayes classifier showed moderate performance on the dataset, with an overall accuracy of around the  0.79. While certain categories had impressive precision and recall, others faced difficulties, highlighting the complexities of text classification tasks.